In [17]:
import os
import pickle
import pandas as pd
import numpy as np
import xgboost as xgb
from calculate_confusion_matrix import calculate_confusion_matrix
import time
from copy import deepcopy
from utils import one_hot_encoder

save_results = True
do_stratify = False

n_boot = 100
split = 0.7
np.random.seed(seed=0)

ft_dir = 'features_long/'

# list feature files
# files = os.listdir(ft_dir)
# files.remove('1515656.dat')
# print('subject 1515656 removed due to spoofed gps.')

# reading top locations
with open('top_locations.dat', 'rb') as f:
    location_top = pickle.load(f)
f.close()

feature_all = []
target_all = []

for filename in files:
    with open(ft_dir+filename, 'rb') as f:  
        feature, target = pickle.load(f)

        # only keeping locations in location_top
        ind = np.array([], int)
        for (i,loc) in enumerate(target['location']):
            if loc in location_top:
                ind = np.append(ind, i)
        feature = feature.loc[ind,:]
        target = target.loc[ind]
        feature = feature.reset_index(drop=True)
        target = target.reset_index(drop=True)
        
        feature_all.append(feature)
        target_all.append(target)
        
    f.close()

confs = []
aucs = []
labels = []
inds = np.arange(0,len(feature_all),1)
inds_split = int(np.floor(split*len(feature_all)))

for i in range(n_boot):
    
    print('------------------')
    print(i)
#     if i==6:
#         print 'subject skipped because of lack of data'
#         continue
    
    # training set
    np.random.shuffle(inds)
    ind_train = inds[:inds_split]
    ind_test = inds[inds_split:]
    
    x_train = pd.concat([feature_all[j] for j in ind_train], axis=0)
    y_train = pd.concat([target_all[j]['location'] for j in ind_train], axis=0)
    x_train = x_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    
    # test set
    x_test = pd.concat([feature_all[j] for j in ind_test], axis=0)
    y_test = pd.concat([target_all[j]['location'] for j in ind_test], axis=0)
    x_test = x_test.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)
    
    # remove foursquare features
#     x_train = x_train.drop(['fsq 0','fsq 1','fsq 2','fsq 3','fsq 4','fsq 5','fsq 6','fsq 7','fsq 8','fsq distance'],axis=1)
#     x_test = x_test.drop(['fsq 0','fsq 1','fsq 2','fsq 3','fsq 4','fsq 5','fsq 6','fsq 7','fsq 8','fsq distance'],axis=1)
#     x_train = x_train.reset_index(drop=True)
#     x_test = x_test.reset_index(drop=True)
    
    # model (sensor)
#     gbm = xgb.XGBClassifier(max_depth=6, n_estimators=50, learning_rate=0.05, nthread=12, subsample=0.25, \
#                         colsample_bytree=0.5, max_delta_step=0, gamma=3, objective='mlogloss', reg_alpha=0.5, \
#                         missing=np.nan)
    # model (sensor + foursquare)
    gbm = xgb.XGBClassifier(max_depth=6, n_estimators=75, learning_rate=0.05, nthread=12, subsample=0.25, \
                        colsample_bytree=0.2, max_delta_step=0, gamma=3, objective='mlogloss', reg_alpha=0.5, \
                        missing=np.nan)
    
    # fitting model
#     gbm.fit(x_train, y_train, eval_set=[(x_train,y_train),(x_test, y_test)], eval_metric='mlogloss', verbose=True)
#     print gbm.evals_result()
    gbm.fit(x_train, y_train)
    
    # training performance
    y_pred = gbm.predict(x_train)
    conf_train, roc_auc_train = calculate_confusion_matrix(y_pred, y_train)

    # test
    y_pred = gbm.predict(x_test)
    conf, roc_auc = calculate_confusion_matrix(y_pred, y_test)
    
    labels.append(np.unique(y_test))
    confs.append(conf)
    aucs.append(roc_auc)

    print(np.unique(y_test))
    print(roc_auc_train, np.nanmean(roc_auc_train))
    print(roc_auc, np.nanmean(roc_auc))

# saving the results
if save_results:
    with open('auc_location_new_10fold3_1515656removed.dat','wb') as f:
        pickle.dump([aucs, confs, labels], f)
    f.close()


------------------
0


/usr/local/lib/python3.4/dist-packages/sklearn/metrics/ranking.py:542: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


["Another's Home" 'Arts & Entertainment' 'Food' 'Home' 'Nightlife Spot'
 'Outdoors & Recreation' 'Professional Or Medical Office' 'Shop Or Store'
 'Spiritual' 'Travel Or Transport' 'Work']
[ 0.82741687  0.95911952  0.78707692  0.93329284  0.97256024  0.99245857
  0.90772608  0.78397938  0.99572816  0.99498771  0.89173315] 0.913279949125
[ 0.81034013  0.49436206  0.73491457  0.89265358  0.99382239         nan
  0.87959814  0.76574339  0.49552124         nan  0.77542529] 0.760264533697
------------------
1


/usr/local/lib/python3.4/dist-packages/sklearn/metrics/ranking.py:542: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


["Another's Home" 'Arts & Entertainment' 'Food' 'Home' 'Nightlife Spot'
 'Outdoors & Recreation' 'Professional Or Medical Office' 'Shop Or Store'
 'Spiritual' 'Travel Or Transport' 'Work']
[ 0.83094852  0.95768594  0.78487327  0.93357627  0.97126222  0.8921896
  0.942048    0.7959013   0.99596014  0.86962113  0.89071657] 0.896798451593
[ 0.64391431  0.74476143  0.69140137  0.86832953  0.99547127         nan
  0.86389996  0.75983148         nan         nan  0.75131746] 0.789865852928
------------------
2


...

/usr/local/lib/python3.4/dist-packages/sklearn/metrics/ranking.py:542: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


["Another's Home" 'Arts & Entertainment' 'Food' 'Home' 'Nightlife Spot'
 'Outdoors & Recreation' 'Professional Or Medical Office' 'Shop Or Store'
 'Spiritual' 'Travel Or Transport' 'Work']
[ 0.84686521  0.94720344  0.79178411  0.93915208  0.95559235  0.97188175
  0.91785756  0.79363181  0.9962479   0.9952821   0.902502  ] 0.914363664797
[ 0.67085634  0.69364341  0.72307513  0.88828609  0.74628253  0.49411674
  0.72925841  0.75039138  0.99496826         nan  0.76158399] 0.745246228829
------------------
98


/usr/local/lib/python3.4/dist-packages/sklearn/metrics/ranking.py:542: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


["Another's Home" 'Arts & Entertainment' 'Food' 'Home' 'Nightlife Spot'
 'Outdoors & Recreation' 'Professional Or Medical Office' 'Shop Or Store'
 'Spiritual' 'Travel Or Transport' 'Work']
[ 0.83069671  0.92234466  0.78159111  0.93689927  0.95662208  0.99191392
  0.94453749  0.7930574   0.99543528  0.99466042  0.89918293] 0.913358297548
[ 0.61764309  0.99339307  0.75686324  0.88558457  0.51770929         nan
  0.85200298  0.73945452         nan         nan  0.79841107] 0.770132728463
------------------
99
["Another's Home" 'Arts & Entertainment' 'Food' 'Home' 'Nightlife Spot'
 'Outdoors & Recreation' 'Professional Or Medical Office' 'Shop Or Store'
 'Spiritual' 'Travel Or Transport' 'Work']
[ 0.83586687  0.89935049  0.77510045  0.9303994   0.99600057  0.94997081
  0.9349086   0.79908111  0.99700257  0.9701879   0.86753033] 0.905036282507
[ 0.66583043  0.71949994  0.72133429  0.86732408         nan  0.49400782
  0.82360018  0.74035078         nan         nan  0.77123727] 0.725398099218


/usr/local/lib/python3.4/dist-packages/sklearn/metrics/ranking.py:542: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


In [13]:
target_all[30]

,location,reason,accomplishment,pleasure,fsq
0,Another's Home,socialize,6.0,6.0,Unknown
1,Home,home,7.0,7.0,Outdoors & Recreation
2,Home,home,0.0,3.0,Outdoors & Recreation
3,Shop Or Store,errand,4.0,4.0,Professional or Medical Office
4,Home,home,6.0,9.0,Outdoors & Recreation
5,Home,home,8.0,7.0,Outdoors & Recreation
6,Home,home,6.0,9.0,Outdoors & Recreation
7,Home,home,2.0,6.0,Outdoors & Recreation
8,Home,home,9.0,7.0,Outdoors & Recreation
9,Home,home,8.0,9.0,Outdoors & Recreation


In [14]:
location_top

array(['Travel Or Transport (Airport, Bus Stop, Train Station, Etc.)',
       'Nightlife Spot (Bar, Club)', 'Spiritual (Church, Temple, Etc.)',
       'Outdoors & Recreation',
       'Arts & Entertainment (Theater, Music Venue, Etc.)', 'Work',
       'Professional Or Medical Office', "Another's Home",
       'Food (Restaurant, Cafe)', 'Home', 'Shop Or Store'], 
      dtype='<U60')